In [16]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image

my_dpi = 192

In [17]:
# Load data and create dataframe
phi = np.load("Phi_201snap.npy")
df = pd.DataFrame(columns=['compression ratio', 'norm'], index=np.arange(201))

for i in range(201):
    arr = phi[:,:,i]

    # Plot image (raw automatically converted to grayscale)
    plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
    plt.imshow(arr, cmap='gray')
    plt.axis("off")

    # Save as PNG & JPG and open JPG again to convert to np array
    plt.savefig('analysis2.1.png', bbox_inches='tight', pad_inches=0)
    plt.savefig('analysis2.1.jpg', bbox_inches='tight', pad_inches=0)
    plt.close()
    im = Image.open('analysis2.1.jpg')
    output_array = np.asarray(im)

    # Create convert function to use to convert output_array back to raw values
    phimax, phimin = np.max(arr), np.min(arr)
    ratio = (phimax-phimin)/255
    def convertRGBToRaw(x):
        return ratio*x + phimin

    # Loop through output_array and replace RGB by raw in copy
    copy = arr.copy()
    for row_index, row in enumerate(output_array):
        for col_index, col in enumerate(row):
            copy[row_index][col_index] = convertRGBToRaw(col[0])

    # Compute norm and compression ratio and add to dataframe
    in_imsize = os.stat('analysis2.1.png').st_size
    out_imsize = os.stat('analysis2.1.jpg').st_size
    df.at[i, 'compression ratio'] = in_imsize/out_imsize
    
    diff = copy - arr
    norm = np.linalg.norm(diff)
    df.at[i, 'norm'] = norm
df

,compression ratio,norm
0,4.148863,0.0
1,4.128169,0.0
2,3.985854,0.0
3,3.891004,0.0
4,3.999041,0.0
...,...,...
196,3.425889,1.414913
197,3.333994,1.557132
198,3.298053,1.722431
199,3.361852,1.913616


In [4]:
df['norm'].isnull().sum() # counting total number of NaN's in df
# np.isnan(np.sum(phi)) # returns true if there's at least one NaN in phi, false otherwise

0

In [5]:
# Computing single norms without for loop
i = 0

my_dpi = 192
phi = np.load("Phi_201snap.npy")
arr = phi[:,:,i] 
plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
plt.imshow(arr, cmap='gray')
plt.axis("off")
plt.savefig('analysis2.1.jpg', bbox_inches='tight', pad_inches=0)
plt.close()
im = Image.open('analysis2.1.jpg')
output_array = np.asarray(im)
output_array
copy = arr.copy()
phimax, phimin = np.max(arr), np.min(arr)
ratio = (phimax-phimin)/255
def convertRGBToRaw(x):
    return ratio*x + phimin
for i, row in enumerate(output_array):
    for j, col in enumerate(row):
        copy[i][j] = convertRGBToRaw(col[0])
norm = np.linalg.norm(copy - arr)
norm

1.3469364304465263e-08